In [ ]:
import time
import math
import socket
import random
import logging
import threading
import matplotlib.pyplot as plt

In [ ]:
class AODV(threading.Thread):

    INF = 999
    weight = {'Power':0.5,'Hop-Count':0.5}
    transfer_loss = {"send":0.005,"receive":0.002}
    transfer_threshold = {"send": 1.5, "receive": 0.5}
    AODV_PATH_DISCOVERY_TIME = 30
    AODV_ACTIVE_ROUTE_TIMEOUT = 30

    def __init__(self,addr):
        threading.Thread.__init__(self)
        self.addr = addr
        self.node_id = "%s:%s"%addr
        self.seq_no = 0
        self.rreq_id = 0
        self.power = 100
        self.counter = 0
        self.sent_bytes = 0
        self.received_bytes = 0
        self.status = "Active"
        self.neighbours = {}
        self.neighbours_of = {}
        self.rreq_id_list = {}
        self.routing_table = {}
        self.message_box = {}
        self.pending_msg_q = {}
        self.sock = socket.socket()
        self.sock.bind(addr)
        self.sock.listen(5)

    def neighbour_of(self,nodes):
        if not isinstance(nodes,list):
            nodes = [nodes]
        for node in nodes:
            s = socket.socket()
            s.connect(node)
            s.send(self.node_id.encode())
            self.neighbours_of['%s:%s'%node] = s
            threading.Thread(target=self.listener,args=(s,)).start()
                    
    def listener(self,sock):
        while True:
            byte = int((self.power-self.transfer_threshold['receive'])/self.transfer_loss['receive'])
            message = sock.recv(byte)
            if message:self.on_recv(message.decode())
    
    def on_recv(self,message):
        # Update params
#         print('Received %s bytes'%len(message))
        self.received_bytes += len(message)
        self.power -= len(message)*self.transfer_loss['receive']
        # Process message
        message = message.split("|")
        switch = {'RREQ_MESSAGE':self.process_rreq,
                  'RREP_MESSAGE': self.process_rrep,
                  'USER_MESSAGE': self.process_user_message}
        try:
            switch[message[0]](message)
        except:
            print('Low Power')
        
    def send(self,sock,message):
        byte = int((self.power-self.transfer_threshold['send'])/self.transfer_loss['send'])
        if len(message)>byte: message = message[:byte]
        # send message
        sock.send(message.encode())
        # Update params
        self.sent_bytes += len(message)
        self.power -= len(message)*self.transfer_loss['send']
#         print('Sent %s bytes'%len(message))

    def restart_route_timer(self, route, create):
        '''Create / Restart the lifetime timer for the given route'''
        if (create == False):
            timer = route['Lifetime']
            timer.cancel()
        timer = threading.Timer(self.AODV_ACTIVE_ROUTE_TIMEOUT, self.route_timeout, [route])
        route['Lifetime'] = timer
        route['Status'] = 'Active'
        timer.start()

    def route_timeout(self, route):
        '''Handle route timeouts'''
        # Remove the route from the routing table
        key = route['Destination']
        self.routing_table.pop(key,None)
        # If the destination is a neighbor, remove it from the neighbor table as well
        if key in self.neighbours:
            self.neighbours.pop(key,None)
        logging.debug("aodv_process_route_timeout: removing " + key + " from the routing table.")

    def path_discovery_timeout(self, node, rreq_id):
        '''Handle Path Discovery timeouts'''
        # Remove the buffered RREQ_ID for the given node
        if node in self.rreq_id_list:
            if rreq_id is self.rreq_id_list[node]:
                self.rreq_id_list.pop(node,None)
                
    def send_rreq(self,dest,dest_seq_no=-1):
        '''Broadcast an RREQ message for the given destination'''
        # Increment our sequence number
        self.seq_no = self.seq_no + 1
        # Increment the RREQ_ID
        self.rreq_id = self.rreq_id + 1
        # Construct the RREQ packet
        message_type = "RREQ_MESSAGE"
        sender = self.node_id
        hop_count = 0
        rreq_id = self.rreq_id
        orig = self.node_id
        orig_seq_no = self.seq_no
        message = message_type + "|" + sender + "|" + str(hop_count) + "|" + str(rreq_id) + "|" + str(dest) + "|" + str(dest_seq_no) + "|" + str(orig) + "|" + str(orig_seq_no)

        # Broadcast the RREQ packet to all the neighbors
        logging.debug("['" + message_type + "', 'Broadcasting RREQ to " + dest + "']")
        for conn in self.neighbours.values():
            self.send(conn,message)
            self.counter += 1

    def process_rreq(self,message):
        '''Process an incoming RREQ message'''
        # Ignore the message if we are not active
        if (self.status == "Inactive"):
            '''
            send rrep with inf hop count
            '''
            self.send_rrep(orig, sender, self.node_id, dest,dest_seq_no,self.INF)
            return
        
        # Extract the relevant parameters from the message
        message_type = message[0]
        sender = message[1]
        hop_count = int(message[2]) + 1
        message[2] = str(hop_count)
        rreq_id = int(message[3])
        dest = message[4]
        dest_seq_no = int(message[5])
        orig = message[6]
        orig_seq_no = int(message[7])
        
        # Check if we are the origin. If we are, discard this RREP.
        if (str(self.node_id) == str(orig)):
            return
        
        logging.debug("['" + message_type + "', 'Received RREQ to " + dest + " from " + sender + "']")
        # After broadcasting this rreq power will be decreases
        power = self.power-len(self.neighbours)*self.transfer_loss['send']
        
        # Add max power availabe if not exist else update
        if len(message) != 9:
            message.append(str(power))
        elif message[8] > str(power):
            message[8] = str(power)

        '''
        Check if have a route to the source. if we have, check whether the weigth for this path 
        is grater than the previous path. If so, update routing table else discard message
        ### path_weight = route['Power']*self.weight['Power']-route['Hop-Count']*self.weight['Hop-Count']
        '''
        if orig in self.rreq_id_list and orig in self.routing_table:
            if rreq_id == self.rreq_id_list[orig]:
                route = self.routing_table[orig]
                # Calculating previous path weight
                prev_path_weight = float(route['Power'])*self.weight['Power']-int(route['Hop-Count'])*self.weight['Hop-Count']
                # Calculating current path weight
                curr_path_weight = float(message[8])*self.weight['Power']-hop_count*self.weight['Hop-Count']
                if curr_path_weight > prev_path_weight:
                    # Update routing table
                    self.routing_table[orig] = {
                        'Destination': str(orig),
                        'Next-Hop': str(sender),
                        'Seq-No': str(orig_seq_no),
                        'Hop-Count': str(hop_count),
                        'Power': message[8],
                        'Status': 'Active'
                    }
                    self.restart_route_timer(self.routing_table[orig], True)
                else:
                    # Discard this RREQ if we have already received this before
                    logging.debug("['RREQ_MESSAGE', 'Ignoring duplicate RREQ (" + orig + ", " + str(rreq_id) + ") from " + sender + "']")
                return

        # This is a new RREQ message. Buffer it first
        self.rreq_id_list[orig] = rreq_id
        
        path_discovery_timer = threading.Timer(self.AODV_PATH_DISCOVERY_TIME,self.path_discovery_timeout, [orig, rreq_id])
        path_discovery_timer.start()
        '''
        Check if we have a route to the source. If we have, see if we need
        to update it. Specifically, update it only if:
        
        1. The destination sequence number for the route is less than the
        originator sequence number in the packet
        2. The sequence numbers are equal, but the hop_count in the packet
        + 1 is lesser than the one in routing table
        3. The sequence number in the routing table is unknown
        
        If we don't have a route for the originator, add an entry
        '''
        if orig in self.routing_table:
            route = self.routing_table[orig]
            # Request come from same origin which already sent rreq request
            if (int(route['Seq-No']) < orig_seq_no):
                route['Seq-No'] = orig_seq_no
                self.restart_route_timer(route, False)
            # Request come from different origin which has same sequence no
            elif (int(route['Seq-No']) == orig_seq_no):
                if (int(route['Hop-Count']) > hop_count):
                    route['Hop-Count'] = hop_count
                    route['Next-Hop'] = sender
                    route['Power'] = message[8]
                    self.restart_route_timer(route, False)
            elif (int(route['Seq-No']) == -1):
                route['Seq-No'] = orig_seq_no
                self.restart_route_timer(route, False)
        else:
            self.routing_table[orig] = {'Destination': str(orig),
                                        'Next-Hop': str(sender),
                                        'Seq-No': str(orig_seq_no),
                                        'Hop-Count': str(hop_count),
                                        'Power': message[8],
                                        'Status': 'Active'}
            self.restart_route_timer(self.routing_table[orig], True) #----------------------------------------------

        # Check if we are the destination. If we are, generate and send an RREP back.
        if (self.node_id == dest):
            self.send_rrep(orig, sender, dest, dest, 0, 0,message[8])
            return
        # We are not the destination. Check if we have a valid route
        # to the destination. If we have, generate and send back an
        # RREP.
        if dest in self.routing_table:
            # Verify that the route is valid and has a higher seq number
            route = self.routing_table[dest]
            status = route['Status']
            route_dest_seq_no = int(route['Seq-No'])
            if (status == "Active" and route_dest_seq_no >= dest_seq_no):
                self.send_rrep(orig, sender, self.node_id, dest, route_dest_seq_no, int(route['Hop-Count']),message[8])
                return
        else:
            # Rebroadcast the RREQ or send RREP with inf hop
            if len(self.neighbours)==1:
                self.send_rrep(orig, sender, self.node_id, dest,dest_seq_no,self.INF, message[8])
            else:
                self.forward_rreq(message)


    def forward_rreq(self,message):
        '''Rebroadcast an RREQ request (Called when RREQ is received by an intermediate node)'''
        msg = message[0] + "|" + self.node_id + "|" + message[2] + "|" + message[3] + "|" + message[4] + "|" + message[5] + "|" + message[6] + "|" + message[7] + "|" + message[8]
        logging.debug("['" + message[0] + "', 'Rebroadcasting RREQ to " + message[4] + "']")
        for conn in self.neighbours.values():
            self.send(conn,msg)

    def send_rrep(self,rrep_dest, rrep_nh, rrep_src, rrep_int_node, dest_seq_no, hop_count,path_power):
        '''Send an RREP message back to the RREQ originator'''
        # Check if we are the destination in the RREP. If not, use the parameters passed.
        if (rrep_src == rrep_int_node):
            # Increment the sequence number and reset the hop count
            self.seq_no = self.seq_no + 1
            dest_seq_no = self.seq_no
            hop_count = 0
        # Construct the RREP message
        message_type = "RREP_MESSAGE"
        sender = self.node_id
        dest = rrep_int_node
        orig = rrep_dest
        message = message_type + "|" + sender + "|" + str(hop_count) + "|" + str(dest) + "|" + str(dest_seq_no) + "|" + str(orig) + "|" + path_power
        # Now send the RREP to the RREQ originator along the next-hop
        self.send(self.neighbours_of[rrep_nh],message)
        logging.debug("['" + message_type + "', 'Sending RREP for " + rrep_int_node + " to " + rrep_dest + " via " + rrep_nh + "']")

    def process_rrep(self,message):
        '''Process an incoming RREP message'''
        # Extract the relevant fields from the message
        message_type = message[0]
        sender = message[1]
        hop_count = int(message[2]) + 1
        message[2] = str(hop_count)
        dest = message[3]
        dest_seq_no = int(message[4])
        orig = message[5]
        path_power = message[6]
        logging.debug("['" + message_type + "', 'Received RREP for " + dest + " from " + sender + "']")
        # Check if we originated the RREQ. If so, consume the RREP.
        if (self.node_id == orig):
            self.counter -= 1
            # Update the routing table. If we have already got a route for
            # this estination, compare the hop count and update the route
            # if needed.
            if (dest in self.routing_table.keys()):
                route = self.routing_table[dest]
                route_hop_count = int(route['Hop-Count'])
                '''Compare best weighted path'''
                # Calculate previous path weight
                prev_path_weight = float(route['Power'])*self.weight['Power']-int(route['Hop-Count'])*self.weight['Hop-Count']
                # Calculate current path weight
                curr_path_weight = float(path_power)*self.weight['Power']-hop_count*self.weight['Hop-Count']
                if curr_path_weight > prev_path_weight:
                    # Update routing table
                    self.routing_table[dest] = {'Destination': dest,
                                            'Next-Hop': sender,
                                            'Seq-No': str(dest_seq_no),
                                            'Hop-Count': str(hop_count),
                                            'Power': path_power,
                                            'Status': 'Active' if not self.counter else 'Processing' }
                    self.restart_route_timer(self.routing_table[dest], True)
            else:
                self.routing_table[dest] = {'Destination': dest,
                                            'Next-Hop': sender,
                                            'Seq-No': str(dest_seq_no),
                                            'Hop-Count': str(hop_count),
                                            'Power': path_power,
                                            'Status': 'Active' if not self.counter else 'Processing'}
                self.restart_route_timer(self.routing_table[dest], True) #------------------------------------------
        else:
            # We need to forward the RREP. Before forwarding, update
            # information about the destination in our routing table.
            if dest in self.routing_table:
                route = self.routing_table[dest]
                '''Compare best weighted path'''
                # Calculate previous path weight
                prev_path_weight = float(route['Power'])*self.weight['Power']-int(route['Hop-Count'])*self.weight['Hop-Count']
                # Calculate current path weight
                curr_path_weight = float(path_power)*self.weight['Power']-hop_count*self.weight['Hop-Count']
                if curr_path_weight > prev_path_weight:
                    # Update routing table
                    self.routing_table[dest] = {'Destination': dest,
                                            'Next-Hop': sender,
                                            'Seq-No': str(dest_seq_no),
                                            'Hop-Count': str(hop_count),
                                            'Power': path_power,
                                            'Status': 'Active'}
                    self.restart_route_timer(self.routing_table[dest], True)
            else:
                self.routing_table[dest] = {'Destination': dest,
                                            'Next-Hop': sender,
                                            'Seq-No': str(dest_seq_no),
                                            'Hop-Count': str(hop_count),
                                            'Power': path_power,
                                            'Status': 'Active'}
                self.restart_route_timer(self.routing_table[dest], True)
            # Now lookup the next-hop for the source and forward it
            route = self.routing_table[orig]
            next_hop = route['Next-Hop']
            self.forward_rrep(message, next_hop)
        
    def forward_rrep(self,message,next_hop):
        '''Forward an RREP message (Called when RREP is received by an intermediate node)'''
        msg = message[0] + "|" + self.node_id + "|" + message[2] + "|" + message[3] + "|" + message[4] + "|" + message[5] + "|" + message[6]
        self.send(self.neighbours_of[next_hop],msg)
        logging.debug("['" + message[0] + "', 'Forwarding RREP for " + message[5] + " to " + next_hop + "']")
        
    def send_user_message(self, message_data,dest):
        '''Send a message to a peer'''
        source = self.node_id
        # Format the message
        message_type = "USER_MESSAGE"
        message = message_type + "|" + source + "|" + dest + "|" + message_data
        # First check if we have a route for the destination
        if dest in self.routing_table.keys():
            # Route already present. Get the next-hop for the destination.
            route = self.routing_table[dest]
            
            if (route['Status'] == 'Inactive'):
                # We don't have a valid route. Broadcast an RREQ.
                self.send_rreq(dest, route['Seq-No'])
            elif (route['Status'] == 'Processing'):
                # Buffer the message and resend it once RREP is received
                self.pending_msg_q[dest] = message_data
            else:
                next_hop = route['Next-Hop']
                self.send(self.neighbours[next_hop],message)
                # update lifetime as the route was used
                self.restart_route_timer(route, False)
                logging.debug("['USER_MESSAGE', '" + source + " to " + dest + " via " + next_hop + "', '" + message_data + "']")
        else:
            # Initiate a route discovery message to the destination
            self.send_rreq(dest, -1)
            # Buffer the message and resend it once RREP is received
            self.pending_msg_q[dest] = message_data
        # send pending message
        while dest in self.pending_msg_q:
            if dest in self.routing_table.keys():
                self.send_user_message(self.pending_msg_q.pop(dest),dest)
                break
                
            
    def process_user_message(self, message):
        '''Process incoming application message'''
        # Get the message contents, sender and receiver
        sender = message[1]
        receiver = message[2]
        msg = message[3]
        # Check if the message is for us
        if (receiver == self.node_id):
            # Add the message to the message box
            self.message_box[sender] = msg
            # Log the message and notify the user
            logging.debug(message)
            print("New message arrived. Issue 'view_messages' to see the contents")
        else:
            # Forward the message by looking up the next-hop. We should have a 
            # route for the destination.
            route = self.routing_table[receiver]
            next_hop = route['Next-Hop']
            message = message[0] + "|" + message[1] + "|" + message[2] + "|" + message[3]
            self.send(self.neighbours[next_hop],message)
            self.restart_route_timer(route, False)
            logging.debug("['USER_MESSAGE', '" + sender + " to " + receiver + "', " + msg + "']")

    def run(self):
        while True:
            conn , _ = self.sock.accept()
            self.neighbours[conn.recv(21).decode()] = conn
            threading.Thread(target=self.listener,args=(conn,)).start()

In [ ]:
class Node(AODV):
    def __init__(self,addr,dist_range=[1,50]):
        super(Node,self).__init__(addr)
        self.position = [random.randint(*dist_range), random.randint(*dist_range)]

In [ ]:
class Network:
    def __init__(self,no_of_node,ip='127.0.0.1',start_port=8000):
        self.no_of_node = no_of_node
        self.nodes = {}
        for i in range(no_of_node):
            addr = (ip,start_port+i)
            self.nodes['%s:%s'%addr] = Node(addr)
            self.nodes['%s:%s'%addr].start()
        # Initilize neighbour
        self.init_neighbour()
            
    def distance(self,src,dest):
        return math.sqrt((self.nodes[src].position[0]-self.nodes[dest].position[0])**2 + \
            (self.nodes[src].position[1]-self.nodes[dest].position[1])**2)

    def init_neighbour(self):
        for node1 in self.nodes:
            for node2 in self.nodes:
                if self.distance(node1,node2) <= self.nodes[node1].power**2 and node1 != node2:
                    self.nodes[node2].neighbour_of(self.nodes[node1].addr)
                    
    def plot_transfer_stat(self,dest):
        transfer = []
        power = []
        for i in [0.0,0.2,0.4,0.6,0.8,1.0]:
            total = 0
            for node in self.nodes:
                self.nodes[node].weight = {'Power':i,'Hop-Count':1-i}
                self.nodes[node].seq_no = 0
                self.nodes[node].rreq_id = 0
                self.nodes[node].power = 50
                self.nodes[node].counter = 0
                self.nodes[node].sent_bytes = 0
                self.nodes[node].received_bytes = 0
                self.nodes[node].status = "Active"
                self.nodes[node].rreq_id_list = {}
                self.nodes[node].routing_table = {}
                self.nodes[node].message_box = {}
                self.nodes[node].pending_msg_q = {}
            self.start_season(dest)
            for name,node in self.nodes.items():
                total += node.received_bytes+node.sent_bytes
            transfer.append(total/self.no_of_node)
            power.append(i)
        plt.plot(power,transfer)
        plt.show()
    
    def start_season(self,dest):
        for node in self.nodes:
            if node != dest:
                self.nodes[node].send_user_message('PING',dest)

In [ ]:
# network = Network(5)
# network.plot_transfer_stat('127.0.0.1:8004')

In [ ]:
class AODV(threading.Thread):

    weight = {'Power':0.5,'Hop-Count':0.5}
    transfer_loss = {"send":0.005,"receive":0.002}
    transfer_threshold = {"send": 1.5, "receive": 0.5}
    BEST_PATH_WAIT_TIME = 2
    MAX_ATTEMPT = 10

    def __init__(self,addr):
        threading.Thread.__init__(self)
        self.addr = addr
        self.node_id = "%s:%s"%addr
        self.seq_no = 0
        self.rreq_id = 0
        self.power = 50
        self.sent_bytes = 0
        self.received_bytes = 0
        self.status = "Active"
        self.neighbours = {}
        self.neighbours_of = {}
        self.rreq_id_list = {}
        self.routing_table = {}
        self.message_box = {}
        self.pending_msg_q = {}
        self.sock = socket.socket()
        self.sock.bind(addr)
        self.sock.listen(20)

    def neighbour_of(self,nodes):
        if not isinstance(nodes,list):
            nodes = [nodes]
        for node in nodes:
            s = socket.socket()
            s.connect(node)
            s.send(self.node_id.encode())
            self.neighbours_of['%s:%s'%node] = s
            threading.Thread(target=self.listener,args=(s,)).start()
                    
    def listener(self,sock):
        while True:
            try:
                message = sock.recv(1024)
                byte = int((self.power-self.transfer_threshold['receive'])/self.transfer_loss['receive'])
                if message:
                    if message[:4] == b'USER':
                        if len(message)<=byte:
                            print('Received %s bytes'%len(message))
                            # Update params
                            self.received_bytes += len(message)
                            self.power -= len(message)*self.transfer_loss['receive']
                            self.on_recv(message.decode())
                        else:
                            print('Low power')
                    else:
                        self.on_recv(message.decode())
            except:
                pass
    
    def on_recv(self,message):
        # Process message
        message = message.split("|")

        switch = {'RREQ':self.process_rreq,
                  'RREP': self.process_rrep,
                  'USER': self.process_user_message}
        
        switch[message[0]](message)
        
    def send(self,sock,message):
        byte = int((self.power-self.transfer_threshold['send'])/self.transfer_loss['send'])
        if message[:4] != 'USER':
            # send message
            sock.send(message.encode())
        else:
            if len(message)<=byte:
                # send message
                sock.send(message.encode())
                # Update params
                self.sent_bytes += len(message)
                self.power -= len(message)*self.transfer_loss['send']
                print('Sent %s bytes'%len(message))
            else:
                print('Low power',self.power)
                
    def send_rreq(self,dest,dest_seq_no=-1):
        '''Broadcast an RREQ message for the given destination'''
        # Increment our sequence number
        self.seq_no = self.seq_no + 1
        # Increment the RREQ_ID
        self.rreq_id = self.rreq_id + 1
        # Construct the RREQ packet
        message_type = "RREQ"
        sender = self.node_id
        hop_count = 0
        rreq_id = self.rreq_id
        orig = self.node_id
        orig_seq_no = self.seq_no
        message = '%s|%s|%s|%s|%s|%s|%s|%s'%(message_type,sender,hop_count,rreq_id,dest,dest_seq_no,orig,orig_seq_no)

        # Broadcast the RREQ packet to all the neighbors
        logging.debug("['" + message_type + "', 'Broadcasting RREQ to " + dest + "']")
        for conn in self.neighbours.values():
            self.send(conn,message)

    def process_rreq(self,message):
        '''Process an incoming RREQ message'''
        # Ignore the message if we are not active
        if (self.status == "Inactive"):
            return
        
        # Extract the relevant parameters from the message
        message_type = message[0]
        sender = message[1]
        hop_count = int(message[2]) + 1
        message[2] = str(hop_count)
        rreq_id = int(message[3])
        dest = message[4]
        dest_seq_no = int(message[5])
        orig = message[6]
        orig_seq_no = int(message[7])
        
        # Check if we are the origin. If we are, discard this RREP.
        if (str(self.node_id) == str(orig)):
            return
        
        logging.debug("['" + message_type + "', 'Received RREQ to " + dest + " from " + sender + "']")
        
        # Add power = 'INF' availabe if not exist
        if len(message) != 9:
            message.append('INF')
            
        power = message[8]

        '''
        Check if have a route to the source. if we have, check whether the weigth for this path 
        is grater than the previous path. If so, update routing table else discard message
        ### path_weight = route['Power']*self.weight['Power']-route['Hop-Count']*self.weight['Hop-Count']
        '''
        if orig in self.rreq_id_list:
            if rreq_id == self.rreq_id_list[orig]:
                route = self.routing_table[orig]
                if route['Status'] == 'Processing':
                    if route['Power'] == 'INF':
                        # Discard this rreq if it is 2nd node because it does not depend on power
                        logging.debug("['RREQ_MESSAGE', 'Ignoring duplicate RREQ (" + orig + ", " + str(rreq_id) + ") from " + sender + "']")
                        return
                    else:
                        print(route['Power'])
                        # Calculating previous path weight
                        prev_path_weight = float(route['Power'])*self.weight['Power']-int(route['Hop-Count'])*self.weight['Hop-Count']
                        # Calculating current path weight
                        curr_path_weight = float(power)*self.weight['Power']-hop_count*self.weight['Hop-Count']
                        if curr_path_weight > prev_path_weight:
                            # Update routing table
                            self.routing_table[orig] = {
                                'Destination': str(orig),
                                'Next-Hop': str(sender),
                                'Seq-No': str(orig_seq_no),
                                'Hop-Count': str(hop_count),
                                'Power': str(power),
                                'Status': 'Processing'
                            }
                        return
                else:
                    # Discard this RREQ if path is active thats mean patn had been established
                    logging.debug("['RREQ_MESSAGE', 'Ignoring duplicate RREQ (" + orig + ", " + str(rreq_id) + ") from " + sender + "']")
                    return

        # This is a new RREQ message. Buffer it first
        self.rreq_id_list[orig] = rreq_id

        self.routing_table[orig] = {'Destination': str(orig),
                                    'Next-Hop': str(sender),
                                    'Seq-No': str(orig_seq_no),
                                    'Hop-Count': str(hop_count),
                                    'Power': str(power),
                                    'Status': 'Processing'}

        best_path_timer = threading.Timer(self.BEST_PATH_WAIT_TIME,self.forward_rreq_or_send_rrep,[message,])
        best_path_timer.start()
                
    def forward_rreq_or_send_rrep(self,message):
        # Extract the relevant parameters from the message
        sender = message[1]
        dest = message[4]
        dest_seq_no = int(message[5])
        orig = message[6]

        # Update routing table
        self.routing_table[orig]['Status'] = 'Active'
        # Update power in message
        try:
            if self.routing_table[orig]['Power'] != 'INF':  
                message[8] = str(min(self.power , float(self.routing_table[orig]['Power'])))
            else:
                message[8] = str(self.power)
        except:
            print(self.routing_table[orig]['Power'])

        # Check if we are the destination. If we are, generate and send an RREP back.
        if (self.node_id == dest):
            self.send_rrep(orig, sender, dest, dest, self.seq_no, 0)
            return
        else:
            # Rebroadcast the RREQ
            self.forward_rreq(message)


    def forward_rreq(self,message):
        '''Rebroadcast an RREQ request (Called when RREQ is received by an intermediate node)'''
        message[1] = self.node_id
        msg = '|'.join(message)
        logging.debug("['" + message[0] + "', 'Rebroadcasting RREQ to " + message[4] + "']")
        for conn in self.neighbours.values():
            self.send(conn,msg)

    def send_rrep(self,rrep_dest, rrep_nh, rrep_src, rrep_int_node, dest_seq_no, hop_count):
        '''Send an RREP message back to the RREQ originator'''
        # Check if we are the destination in the RREP. If not, use the parameters passed.
        if (rrep_src == rrep_int_node):
            # Increment the sequence number and reset the hop count
            self.seq_no = self.seq_no + 1
            dest_seq_no = self.seq_no
            hop_count = 0
        # Construct the RREP message
        message_type = "RREP"
        sender = self.node_id
        dest = rrep_int_node
        orig = rrep_dest
        message = '%s|%s|%s|%s|%s|%s'%(message_type,sender,hop_count,dest,dest_seq_no,orig)
        # Now send the RREP to the RREQ originator along the next-hop
        self.send(self.neighbours_of[rrep_nh],message)
        logging.debug("['" + message_type + "', 'Sending RREP for " + rrep_int_node + " to " + rrep_dest + " via " + rrep_nh + "']")

    def process_rrep(self,message):
        '''Process an incoming RREP message'''
        # Extract the relevant fields from the message
        message_type = message[0]
        sender = message[1]
        hop_count = int(message[2]) + 1
        message[2] = str(hop_count)
        dest = message[3]
        dest_seq_no = int(message[4])
        orig = message[5]
        logging.debug("['" + message_type + "', 'Received RREP for " + dest + " from " + sender + "']")
        # Add in the routing table
        self.routing_table[dest] = {'Destination': dest,
                                    'Next-Hop': sender,
                                    'Seq-No': str(dest_seq_no),
                                    'Hop-Count': str(hop_count),
                                    'Status': 'Active'}
        # Check if we originated the RREQ. If so, consume the RREP.
        if (self.node_id != orig):
            # Now lookup the next-hop for the source and forward it
            route = self.routing_table[orig]
            next_hop = route['Next-Hop']
            self.forward_rrep(message, next_hop)
        
    def forward_rrep(self,message,next_hop):
        '''Forward an RREP message (Called when RREP is received by an intermediate node)'''
        message[1] = self.node_id
        msg = '|'.join(message)
        self.send(self.neighbours_of[next_hop],msg)
        logging.debug("['" + message[0] + "', 'Forwarding RREP for " + message[5] + " to " + next_hop + "']")
        
    def send_user_message(self, message_data,dest):
        '''Send a message to a peer'''
        source = self.node_id
        # Format the message
        message_type = "USER"
        message = '%s|%s|%s|%s'%(message_type,source,dest,message_data)
        # First check if we have a route for the destination
        if dest in self.routing_table.keys():
            # Route already present. Get the next-hop for the destination.
            route = self.routing_table[dest]
            
            if (route['Status'] == 'Inactive'):
                # We don't have a valid route. Broadcast an RREQ.
                self.send_rreq(dest, route['Seq-No'])
            else:
                next_hop = route['Next-Hop']
                self.send(self.neighbours[next_hop],message)
                logging.debug("['USER_MESSAGE', '" + source + " to " + dest + " via " + next_hop + "', '" + message_data + "']")
        else:
            # Initiate a route discovery message to the destination
            self.send_rreq(dest, -1)
            # Buffer the message and resend it once RREP is received
            self.pending_msg_q[dest] = message_data
        # send pending message
        for _ in range(self.MAX_ATTEMPT):
            if dest in self.pending_msg_q:
                if dest in self.routing_table.keys():
                    self.send_user_message(self.pending_msg_q.pop(dest),dest)
                    break
                time.sleep(1)
 
    def process_user_message(self, message):
        '''Process incoming application message'''
        # Get the message contents, sender and receiver
        sender = message[1]
        receiver = message[2]
        msg = message[3]
        # Check if the message is for us
        if (receiver == self.node_id):
            # Add the message to the message box
            self.message_box[sender] = msg
            # Log the message and notify the user
            logging.debug(message)
            print("New message arrived. Issue 'view_messages' to see the contents")
        else:
            # Forward the message by looking up the next-hop. We should have a 
            # route for the destination.
            route = self.routing_table[receiver]
            next_hop = route['Next-Hop']
            message = '|'.join(message)
            self.send(self.neighbours[next_hop],message)
            logging.debug("['USER_MESSAGE', '" + sender + " to " + receiver + "', " + msg + "']")

    def run(self):
        while True:
            try:
                conn , _ = self.sock.accept()
                self.neighbours[conn.recv(21).decode()] = conn
                threading.Thread(target=self.listener,args=(conn,)).start()
            except:
                print('Connection closed\n')
                break

In [ ]:
class Node(AODV):
    def __init__(self,addr,dist_range=[1,100]):
        super(Node,self).__init__(addr)
        self.position = [random.randint(*dist_range), random.randint(*dist_range)]

In [ ]:
class Network:
    def __init__(self,no_of_node,ip='127.0.0.1',start_port=8020):
        self.no_of_node = no_of_node
        self.nodes = {}
        for i in range(no_of_node):
            addr = (ip,start_port+i)
            self.nodes['%s:%s'%addr] = Node(addr)
            self.nodes['%s:%s'%addr].start()
        # Initilize neighbour
        self.init_neighbour()
        
    def shutdown(self):
        for node in self.nodes.values():
            node.sock.close()
            for parent in node.neighbours_of.values():
                parent.close()
            
    def distance(self,src,dest):
        return math.sqrt((self.nodes[src].position[0]-self.nodes[dest].position[0])**2 + \
            (self.nodes[src].position[1]-self.nodes[dest].position[1])**2)

    def init_neighbour(self):
        for node1 in self.nodes:
            for node2 in self.nodes:
                if self.distance(node1,node2) <= self.nodes[node1].power**2 and node1 != node2:
                    self.nodes[node2].neighbour_of(self.nodes[node1].addr)
                    
    def reset(self,factor):
        for node in self.nodes:
            self.nodes[node].weight = {'Power':factor,'Hop-Count':1-factor}
            self.nodes[node].seq_no = 0
            self.nodes[node].rreq_id = 0
            self.nodes[node].power = 50
            self.nodes[node].sent_bytes = 0
            self.nodes[node].received_bytes = 0
            self.nodes[node].status = "Active"
            self.nodes[node].rreq_id_list = {}
            self.nodes[node].routing_table = {}
            self.nodes[node].message_box = {}
            self.nodes[node].pending_msg_q = {}
                    
    def plot_transfer_stat(self,dest):
        transfer = []
        power = []
        for i in [0.0,0.2,0.4,0.6,0.8,1.0]:
            total = 0
            self.reset(i)               
            self.start_season(dest)
            for name,node in self.nodes.items():
                total += node.received_bytes+node.sent_bytes
            transfer.append(total/self.no_of_node)
            power.append(i)
        plt.plot(power,transfer)
        plt.show()
    
    def start_season(self,dest):
        for node in self.nodes:
            if node != dest:
                self.nodes[node].send_user_message('PING',dest)

In [ ]:
network.shutdown()
network = Network(15)

In [ ]:
network.reset(0.0)
network.nodes['127.0.0.1:8020'].send_user_message('PING','127.0.0.1:8024')
network.nodes['127.0.0.1:8021'].send_user_message('PING','127.0.0.1:8024')

In [ ]:
network.reset(0.2)
network.start_season('127.0.0.1:8034')

In [ ]:
network.nodes['127.0.0.1:8034'].message_box

In [ ]:
total = 0     
for name,node in network.nodes.items():
    total += node.received_bytes+node.sent_bytes
total

In [ ]:
network.plot_transfer_stat('127.0.0.1:8034')

In [ ]:
network.plot_transfer_stat('127.0.0.1:8034')